# Demo 6B
The purpose of this demonstration is to reiterate learning objects in lectures 6.2 and 6.4-6.5. After completing this demonstration, you should feel comfortable:
- loading and preparing textual data for analysis with classification-type supervised methods
- training a variety of classifiers and evaluating performance with accuracy, precision, and recall
- fine-tuning classifiers with randomized parameter searches
- evaluating the performance of ensemble models, including a Random Forest and LightGBM model

### The Data, Preprocessing, and Feature Extraction
For this demonstration, we are going to use the same data, preprocessing, and tokenization procedures as in the prior demonstration. The main difference this time, though, is that we are going to focus on *sentiment* instead of *sentiment_score*. Let's look at this column and define a variable for our classification task:

In [1]:
# Load and fix formatting
import pandas as pd, numpy as np, ftfy
data_path = "/storage/ice-shared/mgt8833/classdata/stocktwits_sample.csv.gz"
df = pd.read_csv(data_path)
df = df.replace({np.nan: 'None'}) #ICE loaded these in as "NaN"
df['text'] = df['text'].apply(lambda x: ftfy.ftfy(x))
df

,twitID,author,author_followers,author_ideas,sentiment,text,tickers,timestamp,sentiment_score
0,145610683,shortvolume,564,55841,{'basic': 'Bearish'},$BCOR ranks 1675 by short volume at 72 pct The...,BCOR,2018-11-19 07:40:31,-0.9391
1,142498674,GoodNewsBull,2259,115484,{'basic': 'Bullish'},$NFLX Great Price action. Flush the weak on Op...,NFLX,2018-10-24 13:39:31,-0.2136
2,135730171,livetraderalerts,971,148564,None,$KS 2.5m ago: SEC Current event(s) report - Ot...,KS,2018-08-30 20:22:49,0.0000
3,125482718,WinAllDay,66,1044,{'basic': 'Bullish'},$ADA.X,ADA.X,2018-06-01 16:39:44,0.0000
4,143001861,MBoardman88,6,153,{'basic': 'Bullish'},@Dman20200 just making sure👍. Really trying to...,ATOS,2018-10-27 17:30:41,0.8465
...,...,...,...,...,...,...,...,...,...
9995,141224945,tygerview,3,36,{'basic': 'Bullish'},$CRMD we are almost at $3. So this leads me to...,CRMD,2018-10-12 21:41:38,0.0000
9996,122216361,Berserk74,37,4812,{'basic': 'Bullish'},$CHK chuckie sound like cookie. I like cookie....,CHK,2018-05-03 20:11:21,0.0000
9997,132457414,Nicknar1213,3,358,{'basic': 'Bullish'},$SESN MM's Breaking our Balls.... No worries e...,SESN,2018-08-02 19:25:09,0.8355
9998,137223056,JimInvestor,2588,24975,{'basic': 'Bearish'},Damaged 🚫parabola Crumbling 🔻 $AMD Learn p...,AMD,2018-09-13 19:16:23,-0.6667


As you can see, *sentiment* is missing for some observations. Let's look at the count by unique value of *sentiment*. **PAUSE** and generate those counts:

In [3]:
# Your own work goes here:
df['sentiment'].unique()

array(["{'basic': 'Bearish'}", "{'basic': 'Bullish'}", 'None'],
      dtype=object)

We could use some `loc` commands to create an indicator variable that takes the value 1 for Bullish, 0 for Bearish, and missing for "None". Instead, though, let's use a pandas function that's designed to generate "one hot" encodings, `pd.get_dummies()`:

In [4]:
pd.get_dummies(df['sentiment'])

,None,{'basic': 'Bearish'},{'basic': 'Bullish'}
0,False,True,False
1,False,False,True
2,True,False,False
3,False,False,True
4,False,False,True
...,...,...,...
9995,False,False,True
9996,False,False,True
9997,False,False,True
9998,False,True,False


We can add this to our dataframe with `pd.concat`, remove observations with the value "None" (though keep in another dataframe), and rename the Bullish column to be simply "Y", since that's what we're going to use in our classifiers. 

In [5]:
df2 = pd.concat([df,pd.get_dummies(df['sentiment'])],axis=1)
no_sent = df2.loc[df2['None']==1]
df2 = df2.loc[df2['None']==0].rename(columns = {"{'basic': 'Bullish'}":"Y"})
df2

,twitID,author,author_followers,author_ideas,sentiment,text,tickers,timestamp,sentiment_score,None,{'basic': 'Bearish'},Y
0,145610683,shortvolume,564,55841,{'basic': 'Bearish'},$BCOR ranks 1675 by short volume at 72 pct The...,BCOR,2018-11-19 07:40:31,-0.9391,False,True,False
1,142498674,GoodNewsBull,2259,115484,{'basic': 'Bullish'},$NFLX Great Price action. Flush the weak on Op...,NFLX,2018-10-24 13:39:31,-0.2136,False,False,True
3,125482718,WinAllDay,66,1044,{'basic': 'Bullish'},$ADA.X,ADA.X,2018-06-01 16:39:44,0.0000,False,False,True
4,143001861,MBoardman88,6,153,{'basic': 'Bullish'},@Dman20200 just making sure👍. Really trying to...,ATOS,2018-10-27 17:30:41,0.8465,False,False,True
5,120100505,dhasone225,6,368,{'basic': 'Bullish'},$NFLX My guess is there will be a huge green v...,NFLX,2018-04-16 17:55:35,0.9582,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
9993,134884001,slowslimslider,234,18289,{'basic': 'Bullish'},$SPY Bulls in firm control; no need to fight i...,SPY,2018-08-23 14:38:19,0.6667,False,False,True
9995,141224945,tygerview,3,36,{'basic': 'Bullish'},$CRMD we are almost at $3. So this leads me to...,CRMD,2018-10-12 21:41:38,0.0000,False,False,True
9996,122216361,Berserk74,37,4812,{'basic': 'Bullish'},$CHK chuckie sound like cookie. I like cookie....,CHK,2018-05-03 20:11:21,0.0000,False,False,True
9997,132457414,Nicknar1213,3,358,{'basic': 'Bullish'},$SESN MM's Breaking our Balls.... No worries e...,SESN,2018-08-02 19:25:09,0.8355,False,False,True


We can now move to the rest of our set up procedures. We will keep everything the same as in Demo 6A:
1. Use custom tokenizer that is meant to work with tweets and addresses cash tags. 
2. Remove stop words and perform other normalization / preprocessing steps.
3. Use TF-IDF weighting to extract features. We'll allow for single words and bigrams, but only retain the top 1,000 features since we have less data. I'm also going to limit the matrix to words that appear in no more than 50% of tweets.
4. Split the data into a training and validation set.
The following cells run through this process.

First we'll set up our tokenizer:

In [7]:
# set up tokenizer
import emoji, re
from nltk.tokenize import word_tokenize
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords

stops = stopwords.words('english')
cashtag_rx = re.compile(r'\$[a-z0-9.]+?\b',re.I)

twtokenize = TweetTokenizer()

def myTweetTokenizer(tweet):
    tweet = re.sub(cashtag_rx,'cash_tag',tweet) # get rid of cash tags
    toks = twtokenize.tokenize(tweet)
    good_tokens = []
    for tok in toks:
        if emoji.is_emoji(tok):
            good_tokens.append(tok)
        elif tok=='cash_tag':
            good_tokens.append(tok)        
        elif len(tok)>=3 and tok.isalpha():
            good_tokens.append(tok.lower())
        elif tok.startswith("#"):
            good_tokens.append(tok.lower()) #we'll keep lowercase hashtags    
        else:
            continue

    return good_tokens

Now we'll extract our features (and check the top 10 words):

In [8]:
# Feature Extraction
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(lowercase = False, tokenizer = myTweetTokenizer, ngram_range = (1,2), max_features = 1000, stop_words = stops, max_df=0.50)

dtm = tfidf_vec.fit_transform(df2['text'])

vocab = np.asarray(tfidf_vec.get_feature_names_out())

topn = 10
print(f"{topn} most common words based on word counts:\n")
words = vocab[np.asarray(dtm.todense()).sum(axis=0).argsort()[-topn:][::-1]]
print(", ".join(words))

/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


10 most common words based on word counts:

buy, today, get, cash_tag cash_tag, going, like, good, short, day, time


Finally, we'll split the data:

In [9]:
from sklearn.model_selection import train_test_split

trainX,validX,trainy,validy = train_test_split(dtm, df2['Y'],train_size=0.80,random_state=123)

### Training Simple Classifiers with `sklearn`
We'll begin by training and evaluating performance of two common classifiers for NLP-related tasks, Decision Trees and the Naive Bayes Classifier (NBC). You'll notice the process for training these models is very similar to the approach we used in our regression tasks, though obviously the hyperparameters we set in each classifier vary.
#### Decision Trees ####
For the __[Decision Tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)__, we'll consider the following hyper parameters:
- `criterion`: how the quality of the "split" is measured (or how "good" a given leave is); we will leave this as the default ("gini") for now.
- `max_depth`: how big the tree can get; defaults to `None`, which guarantees an overfit tree. We'll use 200.
- `max_features`: how many features the tree can use. We'll use "sqrt", which uses the square root of the number of features.
- `class_weight`: whether to assign unequal weights to observations. We won't use this yet, but we'll come back to it later.

To evaluate the quality of the fit, there are a number of metrics we can use. We're going to use two functions, `confusion_matrix` and `classification_report`. The former is a contingency table, or 2x2 presentation of Type I and Type II errors. The latter provides detailed information on precision, recall, and the overall F1 score.

Let's train our model!

In [10]:
from sklearn.tree import DecisionTreeClassifier as DTC

tree = DTC(max_depth = 200, max_features = 'sqrt', random_state = 123)
tree.fit(trainX,trainy)

DecisionTreeClassifier(max_depth=200, max_features='sqrt', random_state=123)

The `sklearn` metrics commands accept two position arguments, `y_true` and `y_pred`. The "true" values, `y_true`, are simply the assigned labels for the data we are classifying. The predicted values, `y_pred`, should be generated from our fitted model.

We will now generate the __[`confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)__ and __[`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)__.

In [11]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(trainy,tree.predict(trainX))

cm

array([[ 475,   95],
       [  16, 2563]])

The confusion matrix presents the true values as rows, and predicted values as columns. To illustrate, compute the number of true positives (the bottom row) and compare to the number of true positives in `trainy`.

In [12]:
cm[1,:].sum()==trainy.sum()

True

So, accurate models have lots of observations on the diagonal, and few off diagonal. We can easily compute overall accuracy by comparing the number of observations on the diagonal with the total:

In [13]:
(cm[0,0] + cm[1,1])/cm.sum()

0.9647507145125437

**WOW!!**

Is there a problem?

In [14]:
cm = confusion_matrix(validy,tree.predict(validX))

(cm[0,0] + cm[1,1])/cm.sum()

0.7423857868020305

This model still performs relatively well, though not nearly as good as on the trained data. But that makes sense--decision trees **will** eventually **perfectly** fit the data if given enough leeway. We limited the number of features and depth, so perfect prediction didn't happen, but it was close.

Recall that precision measures the quality of the classification from a "signal to noise" perspective. Recall measures the quality of the model based on it's ability to avoid "false negatives".  Often recall may be more important than precision in a given model, or vice versa. For instance, fire fighters check on pretty much all automated alarm calls (if not cancelled by customer) even though the majority are false alarms. 

Let's compute precision and recall for the 1s in this data:

In [15]:
print(f"Precision is {round((cm[1,1]/cm[:,1].sum())*100,2)}%.") # bottom right divided by total in right column
print(f"Recall is {round((cm[1,1]/cm[1,:].sum())*100,2)}%.") # bottom right divided by total in bottom row

Precision is 81.74%.
Recall is 86.61%.


Why are precision and recall both much higher than overall accuracy? 

The model is **biased**, which we can see if we examine precision and recall for the "0" class (how often was a *bearish* prediction correct):

In [16]:
print(f"Precision is {round((cm[0,0]/cm[:,0].sum())*100,2)}%.") # bottom right divided by total in right column
print(f"Recall is {round((cm[0,0]/cm[0,:].sum())*100,2)}%.") # bottom right divided by total in bottom row

Precision is 36.64%.
Recall is 28.57%.


The confusion matrix is a great way to examine model performance at a very granular level, but we can easily generate metrics we just went through using `sklearn`'s `classification_report`:

In [17]:
print(classification_report(validy,tree.predict(validX)))

              precision    recall  f1-score   support

       False       0.37      0.29      0.32       168
        True       0.82      0.87      0.84       620

    accuracy                           0.74       788
   macro avg       0.59      0.58      0.58       788
weighted avg       0.72      0.74      0.73       788



We've talked about precision, recall, and accuracy (a model level metric). Let's run through the other items on this report:
- `f1-score`: the geometric mean of precision and recall (i.e., sqrt(precision\*recall))
- `support`: the number of observations contributing to the given row (i.e., the number of 0s or 1s, or total)
- `macro avg`: the average value of the given class-level metric
- `weighted avg`: the weighted average value of the given class-level metric (i.e., majority class more heavily influences statistic)

There are a number of ways we can try to deal with model bias, such as resampling (under or oversampling), reducing complexity, or, for some models, using class weights. Class weights default to equal, but if we "value" minority class observations more, then then model will shift parameters towards more accurate classification of those at the expense of the majority class. 

Class weights are a hyperparamter that can be tuned. Let's see what the model looks like if we try to approximate roughly equal weights. which we can do by setting the `class_weight` hyperparameter equal to "balanced".

In [18]:
tree = DTC(max_depth = 200, max_features = 'sqrt', random_state = 123,class_weight='balanced')
tree.fit(trainX,trainy)

DecisionTreeClassifier(class_weight='balanced', max_depth=200,
                       max_features='sqrt', random_state=123)

In [20]:
print(classification_report(validy,tree.predict(validX)))

              precision    recall  f1-score   support

       False       0.36      0.29      0.32       168
        True       0.82      0.86      0.84       620

    accuracy                           0.74       788
   macro avg       0.59      0.57      0.58       788
weighted avg       0.72      0.74      0.73       788



This didn't help at all. We can try more aggressive class weights using a dictionary.

In [21]:
cws = {0:10,1:1}
tree = DTC(max_depth = 200, max_features = 'sqrt', random_state = 123,class_weight=cws)
tree.fit(trainX,trainy)
print(classification_report(validy,tree.predict(validX)))

              precision    recall  f1-score   support

       False       0.27      0.60      0.38       168
        True       0.84      0.57      0.68       620

    accuracy                           0.58       788
   macro avg       0.56      0.58      0.53       788
weighted avg       0.72      0.58      0.62       788



This did slightly change performance, but it doesn't seem to be materially better.

#### Naive Bayes Classifier
Scikit learn offers several implementations of the Naive Bayes (NB) models. For classification, we'll focus on the __[Gaussian NB model](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)__. This model is much simpler to implement since it has few hyperparameters. In fact, there are only 2:
- `priors`: only updated if we have prior probability expectations that differ from the data (we don't)
- `var_smoothing`: a smoothing parameter we'll leave as the default

So let's implement!

In [22]:
from sklearn.naive_bayes import GaussianNB as NBC
nbc = NBC()
nbc.fit(trainX.toarray(),trainy)
print(classification_report(validy,nbc.predict(validX.toarray())))

              precision    recall  f1-score   support

       False       0.24      0.63      0.35       168
        True       0.82      0.46      0.59       620

    accuracy                           0.50       788
   macro avg       0.53      0.55      0.47       788
weighted avg       0.70      0.50      0.54       788



This model did not work well. Precision for the "1s" is pretty good, but otherwise it's not great.

We can try to adjust the `var_smoothing` hyperparameter

In [23]:
nbc = NBC(var_smoothing=1)
nbc.fit(trainX.toarray(),trainy)
print(classification_report(validy,nbc.predict(validX.toarray())))

              precision    recall  f1-score   support

       False       0.36      0.52      0.43       168
        True       0.85      0.75      0.80       620

    accuracy                           0.70       788
   macro avg       0.61      0.64      0.61       788
weighted avg       0.75      0.70      0.72       788



This performs much better now, but like before it's biased. Before moving on, this is an important illustration of the fact that the default hyperparameters are often far from the best choices, which is why model tuning is so important. We'll return to a tuning exercise towards the end of this demo.

#### Stochastic Gradient Descent (SGD)
As discussed in the lectures, SGD is not, itself, a classification technique. Rather, it's an optimization technique that can improve performance of more traditional classification methods. Ones that historically aren't well suited for text can be adapted for an NLP-related task by increasing the efficiency of optimization.

Let's explore how SGD performs using a few different implementations of `sklearn`'s __[SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)__. This classifier implements a variety of "regularized linear models" for classification tasks. Some key parameters include the following:
- `loss`: This defines the loss function to be used, which really controls what type of model you're implementing. For instance `hinge` (the default) uses a support vector machine, `log_loss` implements a logistic regression, and `modified_huber` a third implementation designed for classification tasks. There are several others as well, which we'll look at below.
- `penalty`: This allows you to select the type of regularization penalty to include in the loss function. It defaults to "L2" (which is based on sum of squared feature importances). We'll use "L1" to encourage sparsity of feature selection.

There are many other hyperparameters related to how the model is trained, maximum iterations, tolerances, etc. that we will leave as defaults for now. Let's set up a loop that tries the three primary classification-based loss functions and outputs the model fit statistics. We'll save each model as we go.

In [24]:
from sklearn.linear_model import SGDClassifier as SGD
fitted_models = {}
loss_functions = ['hinge','log_loss','modified_huber', 'squared_hinge', 'perceptron']
for loss in loss_functions:
    sgd = SGD(loss=loss, penalty='l1', n_jobs=-1,random_state=123)
    print(f"Running SGD model with {loss} loss function.")
    sgd.fit(trainX.toarray(),trainy)
    print(f"Classification Report for this model:\n\n")
    print(classification_report(validy,sgd.predict(validX.toarray())))
    fitted_models[loss] = sgd
    print("------------------------------------------------------------")

Running SGD model with hinge loss function.
Classification Report for this model:


              precision    recall  f1-score   support

       False       0.47      0.26      0.33       168
        True       0.82      0.92      0.87       620

    accuracy                           0.78       788
   macro avg       0.64      0.59      0.60       788
weighted avg       0.75      0.78      0.75       788

------------------------------------------------------------
Running SGD model with log_loss loss function.
Classification Report for this model:


              precision    recall  f1-score   support

       False       0.47      0.24      0.31       168
        True       0.82      0.93      0.87       620

    accuracy                           0.78       788
   macro avg       0.64      0.58      0.59       788
weighted avg       0.74      0.78      0.75       788

------------------------------------------------------------
Running SGD model with modified_huber loss function.


Once again, we see that performance is good to very good for classifying bulls, but the bears are much harder to classify. On the one hand, this is to be expected since there's much less data available for those observations (hence the name "minority class"). On the other hand, some of this could be due to overfitting and balanced class weights. We'll do some more robust tuning later.

Since these are inherently linear models, so we can explore the features that are most important in each:

In [25]:
topn = 10
for loss in loss_functions:
    coefs = fitted_models[loss].coef_
    print(f"For the {loss} SGD model, the following {topn} features were most important:\n")
    for c in np.abs(coefs).argsort()[0][::-1][:topn]: # computes absolute values, and argsorts, flips to descending, and grabs first "n"
        print(f"\t{vocab[c]} has a coefficient of {coefs[0][c]}")
    print("\n")

For the hinge SGD model, the following 10 features were most important:

	puts has a coefficient of -6.902199813969005
	🚀 has a coefficient of 6.675852956121767
	cash_tag even has a coefficient of 5.289324050186542
	price target has a coefficient of -5.007159675075177
	loading has a coefficient of 4.996937143541227
	black has a coefficient of -4.956224936444926
	sure has a coefficient of 4.955262327903679
	breakdown has a coefficient of -4.866290421255361
	falling has a coefficient of -4.782478304917979
	lose has a coefficient of -4.744034073990065


For the log_loss SGD model, the following 10 features were most important:

	puts has a coefficient of -9.681424360673088
	lose has a coefficient of -5.726389815340484
	action has a coefficient of 5.471484784929074
	breakdown has a coefficient of -5.422774470177812
	🚀 has a coefficient of 5.418731521757557
	drop has a coefficient of -5.026357519808267
	cash_tag even has a coefficient of 4.944002597325357
	garbage has a coefficient of -4.90

Before moving on, suppose you wanted to see the 5 most positive, and 5 most negative feature importances. How could you adapt the procedure above to print two separate sets of coefficients? **PAUSE** and try to answer that question on your own.

In [26]:
# Your own work goes here:
topn = 5
for loss in loss_functions:
    coefs = fitted_models[loss].coef_
    print(f"For the {loss} SGD model, the following {topn} features had most positive predictive values (bullish):\n")
    for c in coefs.argsort()[0][::-1][:topn]:
        print(f"\t{vocab[c]} has a coefficient of {coefs[0][c]}")
        
    print(f"For the {loss} SGD model, the following {topn} features had most negative predictive values (bearish):\n")
    for c in coefs.argsort()[0][:topn]:
        print(f"\t{vocab[c]} has a coefficient of {coefs[0][c]}")
        
    print("\n")

For the hinge SGD model, the following 5 features had most positive predictive values (bullish):

	🚀 has a coefficient of 6.675852956121767
	cash_tag even has a coefficient of 5.289324050186542
	loading has a coefficient of 4.996937143541227
	sure has a coefficient of 4.955262327903679
	🐃 has a coefficient of 4.678732972173816
For the hinge SGD model, the following 5 features had most negative predictive values (bearish):

	puts has a coefficient of -6.902199813969005
	price target has a coefficient of -5.007159675075177
	black has a coefficient of -4.956224936444926
	breakdown has a coefficient of -4.866290421255361
	falling has a coefficient of -4.782478304917979


For the log_loss SGD model, the following 5 features had most positive predictive values (bullish):

	action has a coefficient of 5.471484784929074
	🚀 has a coefficient of 5.418731521757557
	cash_tag even has a coefficient of 4.944002597325357
	great has a coefficient of 4.325165674008279
	cash_tag day has a coefficient of

### Ensemble Models
Recall that an ensemble model refers to the class of approaches for using estimates from multiple models to form one final prediction. `sklearn` provides classes that you can use to build customized ensembles:
- __[`BaggingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)__: Facilitates setting up a "bag" of weak learners
- __[`AdaBoostClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)__: Facilitates setting up recursive set of "boosted" learners that learn from errors in previous model.
- __[`StackingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier)__: Facilitates setting up stacked model that combines output of first level learners in final predictive model.

These objects are great for customizing your own ensemble model. We're going to explore two "prepackaged" ensemble models, the Random Forest Classifier and LightGBM model.

#### Random Forest
`sklearn`'s `RandomForecastClassifier` (__[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)__) facilitates modeling a random forest, or a collection of "weak" learner decision trees. It has several hyperparameters that require tuning. Many (if not most) of them overlap with the decision tree classifier (e.g., `criterion`, `max_depth`, `max_features`, etc.). 

The main "new" parameters that I focus on are:
- `n_estimators`: the size of the forest, or number of trees. Defaults to 100.
- `max_samples`: The number of observations to use to train each tree (requires `bootstrap` be `True`, which is the default). The default is None (so all samples), but we can set it to something smaller which can help with overfitting.

Let's train a few forests, varying only `n_estimators` and `max_samples` so we can explore how performance and bias changes:

In [27]:
from sklearn.ensemble import RandomForestClassifier as RFC
n_estimators = [50,100,250]
max_samples = [1,0.75,0.25]

for n in n_estimators:
    for m in max_samples:
        rf = RFC(n_estimators=n, max_samples=m, n_jobs = -1, random_state=123, class_weight='balanced')
        print(f"Running Random Forest model with {n} estimators and a max of {round(m*100,2)}% observations per tree.")
        rf.fit(trainX.toarray(),trainy)
        print(f"Classification Report for this model using validation data:\n\n")
        print(classification_report(validy,rf.predict(validX.toarray())))
        print(f"Classification Report for this model using training data:\n\n")
        print(classification_report(trainy,rf.predict(trainX.toarray())))
        print("------------------------------------------------------------")

Running Random Forest model with 50 estimators and a max of 100% observations per tree.
Classification Report for this model using validation data:


              precision    recall  f1-score   support

       False       0.00      0.00      0.00       168
        True       0.79      1.00      0.88       620

    accuracy                           0.79       788
   macro avg       0.39      0.50      0.44       788
weighted avg       0.62      0.79      0.69       788

Classification Report for this model using training data:


              precision    recall  f1-score   support

       False       0.00      0.00      0.00       570
        True       0.82      1.00      0.90      2579

    accuracy                           0.82      3149
   macro avg       0.41      0.50      0.45      3149
weighted avg       0.67      0.82      0.74      3149

------------------------------------------------------------
Running Random Forest model with 50 estimators and a max of 75.0% observati

/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Classification Report for this model using validation data:


              precision    recall  f1-score   support

       False       0.51      0.21      0.30       168
        True       0.82      0.94      0.88       620

    accuracy                           0.79       788
   macro avg       0.66      0.58      0.59       788
weighted avg       0.75      0.79      0.75       788

Classification Report for this model using training data:


              precision    recall  f1-score   support

       False       0.96      0.85      0.90       570
        True       0.97      0.99      0.98      2579

    accuracy                           0.97      3149
   macro avg       0.96      0.92      0.94      3149
weighted avg       0.97      0.97      0.97      3149

------------------------------------------------------------
Running Random Forest model with 50 estimators and a max of 25.0% observations per tree.
Classification Report for this model using validation data:


            

/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Classification Report for this model using validation data:


              precision    recall  f1-score   support

       False       0.51      0.20      0.28       168
        True       0.81      0.95      0.88       620

    accuracy                           0.79       788
   macro avg       0.66      0.57      0.58       788
weighted avg       0.75      0.79      0.75       788

Classification Report for this model using training data:


              precision    recall  f1-score   support

       False       0.95      0.87      0.91       570
        True       0.97      0.99      0.98      2579

    accuracy                           0.97      3149
   macro avg       0.96      0.93      0.94      3149
weighted avg       0.97      0.97      0.97      3149

------------------------------------------------------------
Running Random Forest model with 100 estimators and a max of 25.0% observations per tree.
Classification Report for this model using validation data:


           

/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/storage/ice-shared/mgt8833/class_environment/week6/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Classification Report for this model using validation data:


              precision    recall  f1-score   support

       False       0.50      0.21      0.30       168
        True       0.82      0.94      0.87       620

    accuracy                           0.79       788
   macro avg       0.66      0.58      0.59       788
weighted avg       0.75      0.79      0.75       788

Classification Report for this model using training data:


              precision    recall  f1-score   support

       False       0.96      0.87      0.92       570
        True       0.97      0.99      0.98      2579

    accuracy                           0.97      3149
   macro avg       0.97      0.93      0.95      3149
weighted avg       0.97      0.97      0.97      3149

------------------------------------------------------------
Running Random Forest model with 250 estimators and a max of 25.0% observations per tree.
Classification Report for this model using validation data:


           

That's a lot of output to sift through, but this illustrates two things:
1. There's significant bias in almost all models. In fact, many predict **0** bearish tweets (hence the error messages).
2. Many of the models are severely overfitted. 

#### Light GBM 
The final ensemble we'll train is a the "Light Gradient Boosted Machine" classification model. I chose to include this one because it frequently performs best in NLP tasks. The Python __[implementation of Light GBM](https://lightgbm.readthedocs.io/en/v3.3.5/index.html)__ is not part of `sklearn`, but it has an `sklearn`-like API available through `LGBMClassifier` (__[docs](https://lightgbm.readthedocs.io/en/v3.3.5/pythonapi/lightgbm.LGBMClassifier.html)__). 

If you examine the documentation, you'll see that this model has *many* hyperparameters. For brevity, we'll just implement a model with default parameters except one. `reg_alpha` is the L1 regularization term, which defaults to 0. To encourage sparsity I'm going to make that weight 0.01.

In [28]:
from lightgbm import LGBMClassifier as LGBM

lgbm = LGBM(reg_alpha=0.01,random_state=123,n_jobs=-1)
lgbm.fit(trainX,trainy)

LGBMClassifier(random_state=123, reg_alpha=0.01)

In [29]:
print(classification_report(validy,lgbm.predict(validX.toarray())))

              precision    recall  f1-score   support

       False       0.33      0.08      0.13       168
        True       0.79      0.95      0.87       620

    accuracy                           0.77       788
   macro avg       0.56      0.52      0.50       788
weighted avg       0.70      0.77      0.71       788



Becoming familiar... good overall accuracy, but very biased!

Now that we've trained basic implementations of a wide range of models, let's conclude with a comprehensive tuning process that tries to identify the best model

### Final Model Selection
We've considered X classifiers:
1. Decision Tree
2. Naive Bayes
3. Stochastic Gradient Descent (with several different loss functions)
4. Random Forest
5. Light GBM

We're going to set up a tuning process which re-examines most of these models, but does the following:
1. Considers a wide range of hyperparameter values
2. Uses five-fold cross-validation
3. Optimizes the model on the "macro" f1-score
4. Saves the "best" model

Note that we're going to skip the Naive Bayes classifier since it's a little different in that it has a closed form solution and really only has one parameter to tune.

#### Set up hyperparameter options for each model ###
To start, let's set up our four sets of hyperparameters. Many of these models allow for varying class weights, so we'll start by establishing a set of those to try:

In [30]:
np.random.seed(123)
maj_weight = np.random.uniform(0,0.5,100) # 100 random numbers between 0 and 0.5
class_weights = [{0: 1-k, 1:k} for k in maj_weight]
class_weights.append('balanced') # adds "balanced" as an option
class_weights.append(None) # adds no weighting as an option
class_weights[:10]

[{0: 0.6517654072010692, 1: 0.3482345927989308},
 {0: 0.8569303325248103, 1: 0.14306966747518973},
 {0: 0.8865742732178985, 1: 0.11342572678210155},
 {0: 0.7243426154585544, 1: 0.2756573845414456},
 {0: 0.6402655151072185, 1: 0.35973448489278154},
 {0: 0.7884467699377695, 1: 0.21155323006223048},
 {0: 0.5096179008076922, 1: 0.49038209919230774},
 {0: 0.6575851307075684, 1: 0.34241486929243165},
 {0: 0.7595340492578195, 1: 0.24046595074218047},
 {0: 0.8039412409029247, 1: 0.19605875909707526}]

Now we'll add some model-specific paramters. I've selected up to 5 from each of the model types (usually first 4-5), in addition to class_weights (if available) and a random_state. This is not necessarily comprehensive, but it should allow us to observe a wide range of model configurations. 

We will set up a dictionary for each model:

In [31]:
dt_params = {'criterion':['gini','entropy','log_loss'],
             'max_depth':[None,10, 20, 50, 100],
             'min_samples_split':[2,5,10], # required to split
             'max_features':[None,'sqrt','log2'],
             'class_weight':class_weights,
             'random_state':[123]
            }

sgd_params = {'loss':['hinge','log_loss','modified_huber', 'squared_hinge', 'perceptron'],
              'penalty':['l1','l2','elasticnet',None], # elasticinet is another regularization technique that encourages sparsity
              'alpha':[0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
              'l1_ratio':np.random.uniform(0,1,10).tolist(),
              'random_state':[123],
              'class_weight':class_weights
             }     

rf_params = {'n_estimators':[10,25,50,100,200],
             'criterion':['gini','entropy','log_loss'],
             'max_depth':[None,10, 20, 50, 100],
             'min_samples_split':[2,5,10], # required to split
             'max_features':[None,'sqrt','log2'],
             'max_samples':[None,0.10,0.50,0.75],
             'class_weight':class_weights,
             'random_state':[123]
            }

lgbm_params = {'boosting_type':['gbdt', 'dart', 'goss'],
               'num_leaves':[10,20,30,40,50],
               'max_depth':[-1, 10, 20, 50, 100],
               'learning_rate':[0.01,0.1,0.2,0.5], # note this is a very important parameter, as it controls how much "feedback" the model has. You can also define a function to make it adaptive
               'n_estimators':[25,50,75,100,200],
               'class_weight':class_weights,
               'random_state':[123]
              }

#### Tuning with RandomizedSearchCV
In the previous demonstration, we used a grid search to identify the best set of parameters. If we did a grid search with this set of parameters with 5-fold cross validation we'd have to run... a lot of models! Instead, we will use a *randomized* parameter search with `RandomizedSearchCV` (__[docs](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)__). 

The method for setting up a randomized search is very similar to the grid search we've already done. The main adjustment we have to make is that we'll specify how many parameter combinations to try (since we won't try all). We will also set it up to tune each model and optimize on the macro f1 score.

In [32]:
from sklearn.model_selection import RandomizedSearchCV
final_mods = []
for mod,params in zip([DTC(),SGD(),RFC(),LGBM()],[dt_params, sgd_params, rf_params, lgbm_params]): # zip creates a list of tuples we can use to iterate
    rand_search = RandomizedSearchCV(mod,params, # positional arguments, model and parameter grid
                                     n_iter=25,
                                     scoring='f1_macro', # see options here: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
                                     cv=5,
                                     random_state=123,
                                     n_jobs=-1)
    
    rand_search.fit(trainX,trainy)
    print(f"Classification Report with hold-out sample for best fit of this model:...\n")
    print(type(mod))
    print(classification_report(validy,rand_search.predict(validX.toarray())))
    print("------------------------------------------------------")
    final_mods.append(rand_search)

Classification Report with hold-out sample for best fit of this model:...

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
              precision    recall  f1-score   support

       False       0.32      0.34      0.33       168
        True       0.82      0.81      0.81       620

    accuracy                           0.71       788
   macro avg       0.57      0.57      0.57       788
weighted avg       0.71      0.71      0.71       788

------------------------------------------------------
Classification Report with hold-out sample for best fit of this model:...

<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
              precision    recall  f1-score   support

       False       0.34      0.40      0.37       168
        True       0.83      0.79      0.81       620

    accuracy                           0.71       788
   macro avg       0.59      0.60      0.59       788
weighted avg       0.73      0.71      0.72       788

--------------------

KeyboardInterrupt: 

In [38]:
rand_search = final_mods[0].best_estimator_
print(classification_report(validy,rand_search.predict(validX.toarray())))

              precision    recall  f1-score   support

       False       0.32      0.34      0.33       168
        True       0.82      0.81      0.81       620

    accuracy                           0.71       788
   macro avg       0.57      0.57      0.57       788
weighted avg       0.71      0.71      0.71       788



Overall, all of these classifiers really struggle with the bearish tweets. At this point, I would probably conclude this is simply a noisier class, and it's unlikely we can come up with a model that explains them well. 

However, what if we were in a scenario where getting those right was *much* more important. It may be hard to understand the intuition here, but consider these other scenarios where we have very imbalanced data:
1. Understanding which types of narrative disclosures typically appear in intentionally misstated financial statements
2. Identifying which types of news paper stories lead to significant litigation
3. Understanding which customer complaints result in complete loss of business/revenue.

In all of these cases, the base rate of the scenario we are interested in is fairly low (<1% for intentional misstatements).

Let's use example 3 above and come up with a custom formula. We will do so by quantifying penalties for each type of error we will encounter:
1. Missed 0 (minority class recall): 100 points
2. False 0 (minority class precision): 10 points
3. True 1 (correctly identify majority class): -1 point

We can compute each of these from a confusion matrix and then incorporate the scoring metric into our randomized search. Note that the function we define must "look" like traditional scoring metrics, accepting true `y` first and then `y_pred`.

In [ ]:
def my_scorer(y,y_pred):
    cm = confusion_matrix(y,y_pred)
    crit1 = cm[0,1] # missed 0s woudl be classified as 1s, so first row 2nd column
    crit2 = cm[1,0] # false 0s would be classified as 0s, but actually 1s. so first column 2nd row
    crit3 = cm[1,1] # when we get a "1" right
    return 100*crit1 + 10*crit2 - crit3 # our score

# See how our models we just trained did:
for mod in final_mods:
    print(my_scorer(validy,mod.predict(validX)))

Before incorporating this into our randomized search, we need to make this function a callable scorer, which `sklearn` accomodates with the `make_scorer` function (__[docs](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html)__). There are a few parameters for this function. The only one we need to adjust is whether a higher score is better (`True` is the default).

In [ ]:
from sklearn.metrics import make_scorer

myscorer = make_scorer(my_scorer,greater_is_better = False)

And now we can re-run our search with our new scorer function:

In [ ]:
final_mods2 = []
for mod,params in zip([DTC(),SGD(),RFC(),LGBM()],[dt_params, sgd_params, rf_params, lgbm_params]): # zip creates a list of tuples we can use to iterate
    rand_search = RandomizedSearchCV(mod,params, # positional arguments, model and parameter grid
                                     n_iter=25,
                                     scoring=myscorer,
                                     cv=5,
                                     random_state=123,
                                     n_jobs=-1)
    
    rand_search.fit(trainX,trainy)
    print(f"Classification Report with hold-out sample for best fit of this model:...\n")
    print(type(mod))
    print(classification_report(validy,rand_search.predict(validX.toarray())))
    print("------------------------------------------------------")
    final_mods2.append(rand_search)

Overall model performance declined (by design), but these do appear to do much better with the minority class, at least with respect to recall (which is what had the highest penalty). Let's see what parameters produced the best fit for the LGBM:

In [ ]:
final_mods2[3].best_estimator_

That's all we'll cover on NLP-based classifiers in this module. The concepts will come up again when we discuss deep learning, though.

Before we conclude, I want to highlight that we did not thoroughly examine all means of optimizing the performance of these models. For instance, we could have:
1. Changed the number of features (more or less)
2. Applied dimensionality reduction techniques
3. Standardized or normalized our data (would impact the non-tree based models)
4. Tried resampling (we did not cover this, but oversampling might have improved model performance here by adding synthetic observations to the minority class)

**HOMEWORK**
I'd like each of you to think about a scenario in your own professional experience where a classifier such as the one we trained may be useful--do you expect class imbalance in your scenario? If so, which class likely bears the greatest cost for misclassification? Discuss with the class on Ed Discussions.